[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mnagaku/digraj202103/blob/master/gamemech-analysis.ipynb)

# ゲームメカニクス大全の分析


## 環境準備



パターンの関連を手打ちでdotにしたgamemech.dotを読み込む準備をする。

In [1]:
!sudo apt-get update
!sudo apt-get install -y graphviz libgraphviz-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [30.8 kB]
Ign:3 http://repo.mongodb.org/apt/debian stretch/mongodb-org/3.6 InRelease     
Get:4 http://repo.mongodb.org/apt/debian stretch/mongodb-org/3.6 Release [1,490 B]
Get:6 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]                
Get:7 http://repo.mongodb.org/apt/debian stretch/mongodb-org/3.6 Release.gpg [801 B]
Get:8 http://security.ubuntu.com/ubuntu focal-security InRelease [109 kB]      
Get:9 http://packages.cloud.google.com/apt cloud-sdk InRelease [6,739 B]       
Get:10 http://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [149 kB]
Get:11 http://repo.mongodb.org/apt/debian stretch/mongodb-org/3.6/main amd64 Packages [11.9 kB]
Get:5 https://packages.cloud.google.com/apt kubernetes-xenial InRelease [9,383 B]
Get:12 https://packages.cloud.google.com/apt kubernetes-xenial/main amd64 Packages

In [2]:
!pip install pygraphviz networkx

     |████████████████████████████████| 117 kB 3.4 MB/s eta 0:00:01
  Created wheel for pygraphviz: filename=pygraphviz-1.6-cp38-cp38-linux_x86_64.whl size=212022 sha256=74a68d32e5692ace097315633a71a1fd38e489f0593fa022581f8c34b536e548
  Stored in directory: /home/jovyan/.cache/pip/wheels/dd/37/20/d15e0874c2f2aeb6fb32938b2642fb0cc97ed8c4c247675fca
Successfully built pygraphviz


In [3]:
pip install -U pygraphviz networkx

Requirement already up-to-date: pygraphviz in /opt/conda/lib/python3.8/site-packages (1.6)
Requirement already up-to-date: networkx in /opt/conda/lib/python3.8/site-packages (2.5)
Note: you may need to restart the kernel to use updated packages.


## gamemech.dotの読み込み

書籍に基づいてパターンの関係をgamemech.dotに書き起こした。

In [7]:
res = ! ! ls hoge.dot
if "No such file or directory" in res[0]:
    wget 

get


In [4]:
!cat gamemech.dot

これを分析・加工のために読み込む。

In [9]:
from pygraphviz import *
g = AGraph("gamemech.dot")

## 隣接行列の折り返しと上三角化（正規化）

書き記されたパターンの関係は、関係の両端である2つのパターンの両方に記されているのが理想である。一方のみの記載では、記載のない方からの参照が行えないからである。本書のパターンの関係は、ほぼ一方のみの記載となっており、参照性に問題があると言える。

ただ、データとして扱う上では、無向グラフを隣接行列表現する場合、対角線を挟んだ上三角部と下三角部で同じ情報を二重に持つことになるため、ここではgamemech.dotの隣接情報を上三角部に集約する。

In [10]:
nodes = g.nodes()
edges = g.edges()

for edge in edges:
    if nodes.index(edge[0]) > nodes.index(edge[1]):
        g.delete_edge(edge[0], edge[1])
        if not (edge[1], edge[0]) in edges:
            g.add_edge(edge[1], edge[0])
        else:
            print("("+edge[1]+","+edge[0]+")") # 参照が双方向のものを抽出

g.write("gamemech2.dot")

!dot -Tpng gamemech2.dot -ogamemech2.png

(str03,str07)
(trn01,trn04)
(trn07,trn08)
(trn09,trn10)
(trn14,trn16)
(trn10,act08)
(act02,act08)
(res01,res02)
(res07,res08)
(vic01,vic02)
(vic17,vic18)
(vic15,vic19)
(str03,unc04)
(str07,unc04)
(vic08,unc04)
(unc01,unc04)
(eco10,eco11)
(trn09,eco18)
(eco02,eco18)
(vic04,auc08)
(eco17,auc08)
(auc11,auc15)
(auc11,auc16)
(act10,mov12)
(res12,arc02)
(auc11,arc02)
(vic14,set04)
(unc03,car01)
(act02,car06)
(set05,car06)


関係の参照が双方のものは上記出力の通り。

gamemech2.dotに正規化した内容を出力する。

## 分析対象の検討

本検討では、同一分類（同一Chapter）内

本書では

In [8]:
chapters = {}
for node in g.nodes():
    if node[0:3] in chapters:
        chapters[node[0:3]] += 1
    else:
        chapters[node[0:3]] = 1

chapter_edges = {}
for n1 in range(0,len(chapters)-1):
    n1str = list(chapters)[n1]
    for n2 in range(n1+1,len(chapters)):
        n2str = list(chapters)[n2]
        if not n1str in chapter_edges:
            chapter_edges[n1str] = {}
        chapter_edges[n1str].update({n2str:0})

for edge in g.edges():
    if edge[0][0:3] != edge[1][0:3]:
        chapter_edges[edge[0][0:3]][edge[1][0:3]] += 1

chapg = AGraph(chapter_edges)
chapg.graph_attr['layout']='fdp'

for n1 in range(0,len(chapters)-1):
    n1str = list(chapters)[n1]
    for n2 in range(n1+1,len(chapters)):
        n2str = list(chapters)[n2]
        if chapter_edges[n1str][n2str] == 0:
            chapg.delete_edge(n1str,n2str)
        else:
            chapg.get_edge(n1str,n2str).attr['weight']=chapter_edges[n1str][n2str]

chapg.write("chapg.dot")

!dot -Tpng chapg.dot -ochapg.png

## 同一分類内の関係の洗い出し

In [7]:
label = ""
for edge in g.edges():
    if label != edge[0][0:3]:
        label = edge[0][0:3]
        print("\n"+label)
    if edge[0].startswith(edge[1][0:3]):
        print(edge)


str
('str02', 'str04')
('str02', 'str05')
('str02', 'str07')
('str03', 'str07')

trn
('trn01', 'trn04')
('trn01', 'trn05')
('trn02', 'trn05')
('trn02', 'trn15')
('trn05', 'trn06')
('trn07', 'trn08')
('trn07', 'trn12')
('trn07', 'trn14')
('trn09', 'trn10')
('trn10', 'trn16')
('trn12', 'trn14')
('trn14', 'trn16')
('trn16', 'trn17')

act
('act01', 'act04')
('act01', 'act10')
('act02', 'act08')
('act02', 'act11')
('act03', 'act11')
('act03', 'act13')
('act04', 'act17')
('act05', 'act11')
('act06', 'act07')
('act06', 'act09')
('act07', 'act08')
('act10', 'act11')
('act12', 'act14')
('act13', 'act14')
('act15', 'act16')
('act15', 'act18')

res
('res01', 'res02')
('res01', 'res05')
('res01', 'res07')
('res01', 'res14')
('res01', 'res18')
('res02', 'res05')
('res02', 'res07')
('res02', 'res13')
('res02', 'res21')
('res03', 'res04')
('res07', 'res08')
('res13', 'res14')
('res18', 'res20')

vic
('vic01', 'vic02')
('vic01', 'vic07')
('vic01', 'vic12')
('vic02', 'vic11')
('vic04', 'vic18')
('vic0

同一分類内の関係は、継承（汎化）関係の可能性があり、族の存在が見つかる可能性がある。

## 次数

In [9]:
import networkx as nx
from pprint import pprint

ng = nx.Graph(g)

dg = nx.degree(ng)
dg_sorted = sorted([(d, n) for n, d in dg], reverse=True)
pprint(dg_sorted)

[(18, 'unc01'),
 (15, 'arc02'),
 (11, 'act02'),
 (10, 'unc04'),
 (10, 'car06'),
 (10, 'auc04'),
 (9, 'str02'),
 (9, 'car05'),
 (8, 'trn10'),
 (8, 'mov02'),
 (8, 'eco05'),
 (8, 'car01'),
 (8, 'auc08'),
 (7, 'wpl08'),
 (7, 'wpl05'),
 (7, 'vic18'),
 (7, 'vic01'),
 (7, 'set04'),
 (7, 'res18'),
 (7, 'eco18'),
 (7, 'eco11'),
 (7, 'auc06'),
 (7, 'auc03'),
 (7, 'act15'),
 (6, 'vic11'),
 (6, 'vic07'),
 (6, 'trn05'),
 (6, 'res02'),
 (6, 'res01'),
 (6, 'auc11'),
 (6, 'arc06'),
 (6, 'act08'),
 (6, 'act06'),
 (6, 'act01'),
 (5, 'wpl06'),
 (5, 'vic14'),
 (5, 'vic06'),
 (5, 'unc02'),
 (5, 'trn16'),
 (5, 'trn09'),
 (5, 'trn07'),
 (5, 'trn02'),
 (5, 'str03'),
 (5, 'set02'),
 (5, 'eco13'),
 (5, 'eco02'),
 (5, 'auc13'),
 (5, 'act03'),
 (4, 'unc10'),
 (4, 'str07'),
 (4, 'str05'),
 (4, 'set05'),
 (4, 'res14'),
 (4, 'res07'),
 (4, 'mov04'),
 (4, 'mov01'),
 (4, 'eco16'),
 (4, 'eco14'),
 (4, 'eco01'),
 (4, 'car04'),
 (4, 'auc09'),
 (4, 'auc07'),
 (4, 'auc05'),
 (4, 'auc02'),
 (4, 'arc08'),
 (4, 'act16'),
 (4,

## pagerank

In [10]:
pr = nx.pagerank(ng)
pr_sorted = sorted(pr.items(), key=lambda x:x[1], reverse=True)
pprint(pr_sorted)

[('unc01', 0.02340117260150655),
 ('arc02', 0.020844095505140128),
 ('eco05', 0.015388844735507926),
 ('act02', 0.01476097498977991),
 ('str02', 0.013799148767806406),
 ('car05', 0.013789292236458616),
 ('unc04', 0.013650861466836453),
 ('car06', 0.013639434291487925),
 ('mov02', 0.013067279892646916),
 ('auc04', 0.012770412154343816),
 ('auc08', 0.011776250841981948),
 ('eco11', 0.011577618563098493),
 ('act15', 0.011396060903393603),
 ('vic18', 0.011180077588921405),
 ('car01', 0.011100201417809952),
 ('wpl05', 0.011028895276045421),
 ('set04', 0.010645350896962603),
 ('trn10', 0.010412737415670139),
 ('vic01', 0.010374233936784872),
 ('wpl08', 0.01012896339978216),
 ('vic11', 0.010074762762540157),
 ('auc06', 0.009918781831078215),
 ('trn07', 0.009398492439256329),
 ('eco18', 0.009297289761807302),
 ('act06', 0.009280114418063605),
 ('res18', 0.009268997500335378),
 ('auc03', 0.009201065982372988),
 ('vic07', 0.009131510682552146),
 ('act01', 0.009108553667508122),
 ('res02', 0.0089

## hits

In [11]:
(ht, dummy) = nx.hits(ng)
ht_sorted = sorted(ht.items(), key=lambda x:x[1], reverse=True)
pprint(ht_sorted)

[('unc01', 0.04343290701795568),
 ('act02', 0.028616178481053814),
 ('trn10', 0.02551356901275252),
 ('arc02', 0.02388474712074906),
 ('car06', 0.021982427835356692),
 ('auc04', 0.021470682959568617),
 ('car01', 0.020357005037333807),
 ('unc04', 0.020121313093619725),
 ('trn09', 0.017028037484564166),
 ('res18', 0.016396520375309996),
 ('eco18', 0.01545296248386207),
 ('res01', 0.015217775107350031),
 ('eco16', 0.015133757413383622),
 ('wpl08', 0.01453850731165331),
 ('auc03', 0.014143787921054578),
 ('res14', 0.014004893568045164),
 ('auc13', 0.013835588998984528),
 ('auc11', 0.01350736221455511),
 ('trn05', 0.013303622923629603),
 ('auc05', 0.013008703176993498),
 ('auc06', 0.012941087699963646),
 ('auc07', 0.012883997366095268),
 ('act08', 0.012587272347650637),
 ('vic01', 0.012390726941699862),
 ('vic07', 0.012093080214990615),
 ('unc08', 0.011682767629749851),
 ('res08', 0.01142732642449925),
 ('wpl06', 0.011405055526570597),
 ('wpl05', 0.01123828061614407),
 ('unc02', 0.011049482

res03、04の値的には、pagerankは微妙。ただ、hitsもエッジ本数稼いだのが強く出過ぎてる気がする。パラメータで調整かけるか、自分で固有ベクトル求める系計算するか。